In [1]:
%load_ext autoreload
%autoreload 2


In [69]:
with open("../data/small_train.txt", "r") as f:
    f.readline() # the first line is just "text"
    data = f.read()
    
# first 10000 words
data = " ".join(data.split()[:10000])
    
data = data.lower()
data = data.replace(".", "")
data = data.replace(",", "")
data = data.replace("!", "")
data = data.replace("?", "")
data = data.replace("'", "")
data = data.replace("\"", "")

word_to_ids = {word: i for i, word in enumerate(set(data.split()))}
ids_to_word = {i: word for word, i in word_to_ids.items()}

In [70]:
tokenized_data = [word_to_ids[word] for word in data.split()]
untokenized_data = data.split()

In [72]:
from utils import NGramTrie

trie = NGramTrie.fit(tokenized_data, ngram_max_length=3)

In [75]:
trie.search([word_to_ids[word] for word in "one day".split()]), " ".join(untokenized_data).count("one day")

(56, 56)

In [76]:
trie.search([word_to_ids[word] for word in "one day a".split()]), " ".join(untokenized_data).count("one day a")

(11, 11)

In [78]:
trie.search([word_to_ids[word] for word in "one day a".split()], "++*"), " ".join(untokenized_data).count("one day a")

(56, 11)